In [16]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
import json

_ = load_dotenv(find_dotenv())

client = OpenAI()

## 1. Create an Assistant

In [17]:
assistant = client.beta.assistants.create(
    name="Javascrit Teacher",
    instructions="You are Javascript teacher. Write and run code to answer queries",
    tools=[{"type":"code_interpreter"}],
    model="gpt-3.5-turbo-1106",
)

assistant # print

Assistant(id='asst_1ilSkEoLfaVokOPAJpDSbgUx', created_at=1703350061, description=None, file_ids=[], instructions='You are Javascript teacher. Write and run code to answer queries', metadata={}, model='gpt-3.5-turbo-1106', name='Javascrit Teacher', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])

## 2. Create a Thread

In [18]:
thread = client.beta.threads.create()

print(thread)

thread_message = client.beta.threads.messages.list(thread.id)

thread_message # there is no message right now

Thread(id='thread_FDGVbFrZA9c74xyoiRDtkbJs', created_at=1703350061, metadata={}, object='thread')


SyncCursorPage[ThreadMessage](data=[], object='list', first_id=None, last_id=None, has_more=False)

## 3. Add a Message to a Thread

In [19]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are the primitive data types?"
)

message

ThreadMessage(id='msg_NYuj6FULrFU3hVAtFAPhewnc', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What are the primitive data types?'), type='text')], created_at=1703350062, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_FDGVbFrZA9c74xyoiRDtkbJs')

### List the Message in a Thread

In [20]:
thread_message = client.beta.threads.messages.list(thread.id)

thread_message # now message has been appended to thread

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_NYuj6FULrFU3hVAtFAPhewnc', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What are the primitive data types?'), type='text')], created_at=1703350062, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_FDGVbFrZA9c74xyoiRDtkbJs')], object='list', first_id='msg_NYuj6FULrFU3hVAtFAPhewnc', last_id='msg_NYuj6FULrFU3hVAtFAPhewnc', has_more=False)

## 4. Run the Assistant

In [21]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    # instructions="This student concepts are weak"
        # this is optional, it overrides the Assistant's default instruction
)

run

Run(id='run_wcLlKKLEMXv8QpqIp0PqvYgi', assistant_id='asst_1ilSkEoLfaVokOPAJpDSbgUx', cancelled_at=None, completed_at=None, created_at=1703350063, expires_at=1703350663, failed_at=None, file_ids=[], instructions='You are Javascript teacher. Write and run code to answer queries', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_FDGVbFrZA9c74xyoiRDtkbJs', tools=[ToolAssistantToolsCode(type='code_interpreter')])

## 5. Check the Run Assistant

In [22]:
# by default Run goes into 'queued' status

run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)

run

Run(id='run_wcLlKKLEMXv8QpqIp0PqvYgi', assistant_id='asst_1ilSkEoLfaVokOPAJpDSbgUx', cancelled_at=None, completed_at=None, created_at=1703350063, expires_at=1703350663, failed_at=None, file_ids=[], instructions='You are Javascript teacher. Write and run code to answer queries', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1703350063, status='in_progress', thread_id='thread_FDGVbFrZA9c74xyoiRDtkbJs', tools=[ToolAssistantToolsCode(type='code_interpreter')])

## 6. Display the Assistant's Response

In [27]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

print(messages)
print(messages.data)

print("")
# displaying in conversation style
for message in reversed(messages.data):
  # revesed() because assistant message was appended and it is listed first.
  print(message.role + ": " + message.content[0].text.value)

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_yZNCW0ibzTNTnF8E2wmzCpIj', assistant_id='asst_1ilSkEoLfaVokOPAJpDSbgUx', content=[MessageContentText(text=Text(annotations=[], value='The primitive data types in JavaScript are:\n\n1. Number: Represents both integer and floating-point numbers.\n2. String: Represents a sequence of characters.\n3. Boolean: Represents true or false values.\n4. Undefined: Represents a variable that has been declared but has not been assigned a value.\n5. Null: Represents an intentional absence of any object value.\n6. Symbol (added in ECMAScript 6): Represents a unique and immutable value and may be used as the key of an Object property.\n\nThese primitive data types are immutable, meaning they cannot be changed.'), type='text')], created_at=1703350064, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_wcLlKKLEMXv8QpqIp0PqvYgi', thread_id='thread_FDGVbFrZA9c74xyoiRDtkbJs'), ThreadMessage(id='msg_NYuj6FULrFU3hVAtFAPhewn